In [0]:
spark.conf.set(
    "fs.azure.account.key.bp040storage.dfs.core.windows.net",
    "kiENG28kPnyzB+dA3xwAxlcWC5eHv56ldDJidJPlneDLglk3zhWyPkLZL+TOOzfqiyFn7i9Ys7x3+AStxKkaaQ==")

In [0]:
display(dbutils.fs.ls("abfss://bp040container@bp040storage.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://bp040container@bp040storage.dfs.core.windows.net/raw/,raw/,0,1729970244000


In [0]:
df = spark.read.csv("abfss://bp040container@bp040storage.dfs.core.windows.net/raw/population_by_age.tsv", header=True,sep="\t",inferSchema="true");
df.show(5)

+-----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|indic_de,geo\time|2008 |2009 |2010 |2011 |2012 |2013 |2014 |2015 |2016 |2017 |2018 |2019 |
+-----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|      PC_Y0_14,AD|14.6 |14.5 |14.5 |15.5 |15.5 |15.5 |   : |   : |   : |   : |   : |13.9 |
|      PC_Y0_14,AL|24.1 |23.3 |22.5 |21.6 |20.7 |20.1 |19.6 |19.0 |18.5 |18.2 |17.7 |17.2 |
|      PC_Y0_14,AM|19.0 |18.6 |18.3 |   : |   : |   : |   : |19.4 |19.6 |20.0 |20.2 |20.2 |
|      PC_Y0_14,AT|15.4 |15.1 |14.9 |14.7 |14.6 |14.4 |14.3 |14.3 |14.3 |14.4 |14.4 |14.4 |
|      PC_Y0_14,AZ|23.2 |22.6 |22.6 |22.3 |22.2 |22.3 |22.4 |22.4 |22.5 |22.6 |22.6 |22.4 |
+-----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
only showing top 5 rows



In [0]:
df_population = df.withColumnRenamed('indic_de,geo\\time', 'Geo_Time')
df_population.show(5)

+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|   Geo_Time|2008 |2009 |2010 |2011 |2012 |2013 |2014 |2015 |2016 |2017 |2018 |2019 |
+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|PC_Y0_14,AD|14.6 |14.5 |14.5 |15.5 |15.5 |15.5 |   : |   : |   : |   : |   : |13.9 |
|PC_Y0_14,AL|24.1 |23.3 |22.5 |21.6 |20.7 |20.1 |19.6 |19.0 |18.5 |18.2 |17.7 |17.2 |
|PC_Y0_14,AM|19.0 |18.6 |18.3 |   : |   : |   : |   : |19.4 |19.6 |20.0 |20.2 |20.2 |
|PC_Y0_14,AT|15.4 |15.1 |14.9 |14.7 |14.6 |14.4 |14.3 |14.3 |14.3 |14.4 |14.4 |14.4 |
|PC_Y0_14,AZ|23.2 |22.6 |22.6 |22.3 |22.2 |22.3 |22.4 |22.4 |22.5 |22.6 |22.6 |22.4 |
+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import *

df_population = df_population.withColumn('age_group',regexp_replace(split(df_population['Geo_Time'],',')[0],'PC_','')).withColumn('country_code',split(df_population['Geo_Time'],',')[1])
df_population.show(5)


+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+---------+------------+
|   Geo_Time|2008 |2009 |2010 |2011 |2012 |2013 |2014 |2015 |2016 |2017 |2018 |2019 |age_group|country_code|
+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+---------+------------+
|PC_Y0_14,AD|14.6 |14.5 |14.5 |15.5 |15.5 |15.5 |   : |   : |   : |   : |   : |13.9 |    Y0_14|          AD|
|PC_Y0_14,AL|24.1 |23.3 |22.5 |21.6 |20.7 |20.1 |19.6 |19.0 |18.5 |18.2 |17.7 |17.2 |    Y0_14|          AL|
|PC_Y0_14,AM|19.0 |18.6 |18.3 |   : |   : |   : |   : |19.4 |19.6 |20.0 |20.2 |20.2 |    Y0_14|          AM|
|PC_Y0_14,AT|15.4 |15.1 |14.9 |14.7 |14.6 |14.4 |14.3 |14.3 |14.3 |14.4 |14.4 |14.4 |    Y0_14|          AT|
|PC_Y0_14,AZ|23.2 |22.6 |22.6 |22.3 |22.2 |22.3 |22.4 |22.4 |22.5 |22.6 |22.6 |22.4 |    Y0_14|          AZ|
+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+---------+------------+
only showing top 5 

In [0]:
df_population.write.mode("overwrite").partitionBy("country_code","age_group",).format("parquet").save("abfss://processed1@bp040storage.dfs.core.windows.net/population/partition/populationc_country_code")